<a href="https://colab.research.google.com/github/ramselvaraj/F1-Twitter-Sentiment-Analysis/blob/main/twitterScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://img.theweek.in/content/dam/week/news/sports/images/2021/12/13/hamilton-verstappen-formula-one-abu-dhabi-f1-twitter.jpg" height = 300 width = 500></img>
# **Building the Dataset for F1 Twitter Sentiment Analysis**
## Constructing Tweet Dataset of tweets following Abu Dhabi 2021 GP
In this project, I am going to be constructing a dataset containing Tweet data following the controversial title-deciding Abu Dhabi 2021 GP 
<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

MessageError: ignored

In [ ]:
!pip install better_profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.3 MB/s 


In [ ]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tweepy
import pandas as pd
import numpy as np
import re
from wordcloud import WordCloud
from better_profanity import profanity
from textblob import TextBlob
import matplotlib.pyplot as plt 

In [ ]:

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

This function requires '**Elevated**' access, so make sure you apply for that on the Twitter Developer Portal

In [ ]:
user = api.verify_credentials()

In [ ]:
print("ID: ",user.id)
print("UserName: ",user.screen_name)
print("Follower Count: ",user.followers_count)
print("Account Created: ",user.created_at)


ID:  1526576216052224000
UserName:  ramsel84
Follower Count:  10
Account Created:  2022-05-17 14:52:05


My account has successfully been verified.

We will be building a dataset with replies to a tweet by the official F1 account announcing that Redbull is now the constructor's champion

In [ ]:
import datetime
# user_name = 'redbullracing'
# tweet_id = '1584285187638431745'
user_name = 'F1'
tweet_id = '1584285589678960640'
replies = []

for tweet in tweepy.Cursor(api.search,q='to:'+user_name , result_type = 'mixed', timeout=999999).items(1500):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            #print(tweet.in_reply_to_status_id_str)
            replies.append(str(tweet.text))
replies


['@F1 @redbullracing #Cheaters']

In [ ]:
replies_cur = replies

In [ ]:
repliesfinal = pd.DataFrame(replies_cur)
repliesfinal.to_csv('repliesTemp.csv')

## Cleaning the data we have just obtained

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML/F1TwitterSentimentAnalysis/repliesTemp.csv')
df = df[['text']]
df.head()

NameError: ignored

In [ ]:
#cleaning the '@redbullracing' mentions
df['text'] = df['text'].str[14:]
df.head()

,text
0,congrats all! 👏
1,Cheats do prosper it seems 🤦‍♂️
2,We are the champions!!! @SChecoPerez Grandeee...
3,It’s a shame that even Horner can’t admit the...
4,Lets fucking gooooooo!!!!!!! https://t.co/Ecf...


In [ ]:
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    r = tweet.lower()
    r = profanity.censor(r)
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

In [ ]:
cleaned = [clean_tweet(tw) for tw in df['text']]
cleaned[0:5] #showing first 5 elements

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


['congrats all',
 'cheats do prosper it seems',
 'we are champions grandeeee el mejor leyenda una bestia',
 'it s shame that even horner can t admit over spend keep sport disgrace',
 'lets gooooooo']

In [ ]:
df['text'] = cleaned
df = df[df.text != '']
df.shape

(267, 1)

In [ ]:
df.head()

,text
0,congrats all
1,cheats do prosper it seems
2,we are champions grandeeee el mejor leyenda un...
3,it s shame that even horner can t admit over s...
4,lets gooooooo


In [ ]:
df.to_csv('repliesFinal.csv')